In [1]:
import numpy as np
import pandas as pd

In [2]:
read_csv = pd.read_csv("/Users/jooh8/Documents/GitHub/Quant/data/A005930/Index_A005930.txt", sep=" ", low_memory=False)

In [3]:
len(read_csv)

10594

In [4]:
for period in range(10):
    
    read_csv["고가_{}".format(period)] = read_csv["고가"].shift(periods=period).fillna(0)

In [5]:
for period in range(10):
    
    read_csv["최고가_{}".format(period)] = read_csv[["고가_{}".format(i) for i in range(period + 1)]].apply(np.max, 1)

In [6]:
for period in range(10):
    
    read_csv["Target_{}".format(period)] = read_csv["시가"] * 1.05 < read_csv["고가_{}".format(period)]

In [7]:
columns = list(read_csv.columns)

In [8]:
momentums = columns[11:-30]
print(momentums)

['A Ratio_20', 'B Ratio_20', 'Aroon_20 Up', 'Down', 'Aroon Osillator_20', 'ATR_14', 'BB-RSI_종가_10', 'BB-RSI_20_2.00 상한', 'BB-RSI 하한', 'BPDL RSI_14', 'BPDL Stochatic_14', 'Chande Momentum Oscillator_10', "Chaikin's Volatility_14,14", 'CompuTrac Volatility_10', '+DI_14', '-DI_14', 'ADX_14', 'ADXR_14', '+DI(simple)_14', '-DI(simple)_14', 'ADX(simple)_14,14', 'ADXR(simple)_14,14,14', 'Energy+_14', 'Energy-_14', 'High Low Envelope_1', 'High Low Oscillator_3', 'Inertia_20,14,10', 'Klinger Oscillator', 'Linear Trend Oscillator_10,20', 'LRS_종가,14', 'Mass Index_20,9', 'Morris Mixed Momentum', 'New BPDL HiLo Index MA_14,7', 'Open Difference_15', 'QStick_14', 'Random Walk Index_15,3', 'RCI_5', 'RCI_9', 'RCI_13', 'RCI_18', 'Relative Volatility Index_단순,14,10', 'Reverse 단기_12', ' 장기_24', 'RSI_종가,14', 'RSI(simple)_종가,14', 'Sigma_종가,20', 'SMI_5,3,3', 'Standard Deviation_14,2.00', 'Standard Error_종가,14', 'Fast %K_5', 'Fast %D_3', 'Slow %K_5,3', 'Slow %D_3', 'Slow(Simple) %K_5,3', 'Slow(Simple) %D_3', 

In [9]:
targets = columns[-10:]
print(targets)

['Target_0', 'Target_1', 'Target_2', 'Target_3', 'Target_4', 'Target_5', 'Target_6', 'Target_7', 'Target_8', 'Target_9']


In [10]:
X_train = read_csv[-2000:-100][momentums]
Y_train = read_csv[-2000:-100][targets]

In [11]:
X_test = read_csv[-100:][momentums]
Y_test = read_csv[-100:][targets]

In [12]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
import numpy as np

In [13]:
scaler = preprocessing.StandardScaler().fit(X_train)

C:\Users\jooh8\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)


In [14]:
forest = RandomForestClassifier(n_estimators=100, random_state=1)

In [15]:
multi_target_forest = MultiOutputClassifier(forest, n_jobs=-1)

In [16]:
multi_target_forest.fit(scaler.transform(X_train), Y_train)

C:\Users\jooh8\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=1, verbose=0, warm_start=False),
           n_jobs=-1)

In [17]:
predict = multi_target_forest.predict(scaler.transform(X_train))

C:\Users\jooh8\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


In [19]:
predict = multi_target_forest.predict(scaler.transform(X_test))

C:\Users\jooh8\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


In [21]:
corr = predict == Y_test

In [24]:
Y_test.apply(np.sum, 0)

Target_0     0
Target_1     0
Target_2     5
Target_3    11
Target_4    12
Target_5    16
Target_6    16
Target_7    16
Target_8    23
Target_9    25
dtype: int64

In [23]:
corr.apply(np.mean, 0)

Target_0    1.00
Target_1    1.00
Target_2    0.95
Target_3    0.90
Target_4    0.94
Target_5    0.94
Target_6    0.96
Target_7    0.93
Target_8    0.94
Target_9    0.95
dtype: float64